# Parameters

In [1]:
batch_size = 32
dataset_name = 'plantVillage-test-mohanty'
dataset_path = '../datasets/' + dataset_name
model_diagram_path = 'vgg_model_diagram.png'
features_save_path = 'features.h5'

buffer_size = 1000

input_width = 224
input_height = 224

output_width = 7
output_height = 7
output_depth = 512


import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))



Classes:  ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
3


# Initialize model

In [3]:
import os, sys
# import pydot_ng as pydot
# pydot.find_graphviz()
# os.environ.get('PATH', '')

# remove these
# sys.path.append('..')
# remove these

from keras.utils.vis_utils import plot_model
from keras.applications import InceptionV3
from keras.optimizers import SGD

model = InceptionV3(weights="imagenet", include_top=False)

model.summary()
plot_model(model, to_file=model_diagram_path, show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])



[0]              
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, None, None, 1 147456      average_pooling2d_7[0][0]        
__________________________________________________________________________________________________
batch_normalization_61 (BatchNo (None, None, None, 1 576         conv2d_61[0][0]                  
__________________________________________________________________________________________________
batch_normalization_64 (BatchNo (None, None, None, 1 576         conv2d_64[0][0]                  
__________________________________________________________________________________________________
batch_normalization_69 (BatchNo (None, None, None, 1 576         conv2d_69[0][0]                  
__________________________________________________________________________________________________
batch_normalization_70 (BatchNo (None, None, None, 1 576         conv2d_70[0][0]           

# Load dataset

In [2]:
import sys, os
sys.path.append('..')
import numpy as np
from imutils import paths
import random


from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# Get the list of image paths
image_paths = list(paths.list_images(dataset_path))
random.shuffle(image_paths)

#extract labels
labels = [p.split(os.path.sep)[-2] for p in image_paths]

Using TensorFlow backend.


# Encode labels and split dataset

In [4]:
from sklearn.preprocessing import LabelEncoder



# Convert labels in integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Initialize Datsetwriter

In [7]:
import utils
from utils.io.hdf5datasetwriter import HDF5DatasetWriter

dataset = HDF5DatasetWriter((len(image_paths), output_width * output_height * output_depth),
            features_save_path, dataKey="features", bufSize=buffer_size)

# Store the classes to to file
dataset.storeClassLabels(le.classes_)

# Forward propagate through model

In [8]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications import imagenet_utils

print(image_paths)

# loop over the image_paths in batches
for i in np.arange(0, len(image_paths), batch_size):
  # extract the image paths and labels for the current batch,
    batchPaths = image_paths[i:i + batch_size]
    batchLabels = labels[i:i + batch_size]
  # initializes a list to store the images about to be loaded and fed into VGG16.
    batchImages = []
    print(batchImages)

    # loop over the images and labels in the current batch
    for (j, imagePath) in enumerate(batchPaths):

       # load the images and convert to correct size.
        image = load_img(imagePath, target_size=(224, 224))
        # convert to numpy array
        image = img_to_array(image)

        # Expand dims and apply preprocessing
        # (1) expanding the dimensions
        # (2) subtracting the mean RGB pixel intensity from ImageNet dataset
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)

    # add the image to the batch
        batchImages.append(image)

  #  .vstack method used to “vertically stack”  images such that they have the shape
    # (N, 224, 224, 3) where N is the size of the batch.
    batchImages = np.vstack(batchImages)
    # pass the images through the network and use the outputs as our actual features
    features = model.predict(batchImages, batch_size=bs)

    # Output of VGG16's final Max Pool layer is (N, 512, 7, 7)
    # Flatten them so that they are of shape (N, 512 * 7 * 7)
    features = features.reshape((features.shape[0], 7 * 7 * 512))

  # add the features and labels to our HDF5 dataset
    dataset.add(features, batchLabels)
    pbar.update(i)

dataset.close()
pbar.finish()

e7c-8ff2e323e71d___GCREC_Bact.Sp 3533.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Bacterial_spot/b0049dbf-cdef-4a85-b6e1-b37ed6bc4cfa___UF.GRC_BS_Lab Leaf 0696.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Early_blight/d4a47a64-0c10-4a43-8712-4eee11c8cf1b___RS_Erly.B 9524.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Bacterial_spot/bac40427-ce12-4369-8866-9e48e2f982da___GCREC_Bact.Sp 6092.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Early_blight/eabc47ee-5023-476b-9992-53baa566d270___RS_Erly.B 7757.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Bacterial_spot/e7c9c9ed-9e51-4056-93c7-f1ddf74f2ebc___UF.GRC_BS_Lab Leaf 0945.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Bacterial_spot/f6084490-18e3-489a-9144-4584db9c1d38___GCREC_Bact.Sp 3049.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Early_blight/aea7d3be-648a-4e1b-932a-b0058b085a7a___RS_Erly.B 9517.JPG', '../datasets/plantVillage-test-mohanty/Tomato___Bacterial_spot/e99d601c-2ff

NameError: name 'bs' is not defined